In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/new-road-400m/train_df_with_road_density (2).csv
/kaggle/input/new-road-400m/valid_df_with_road_density (2).csv
/kaggle/input/data-with-density-200m/validation_data_with_density.csv
/kaggle/input/data-with-density-200m/uncorrupted_training_data_with_density.csv
/kaggle/input/building-density3-15-20m-200m/train_df_with_building_density3 (6).csv
/kaggle/input/building-density3-15-20m-200m/valid_df_with_building_density3 (6).csv
/kaggle/input/building-density2-500m-v2/valid_df_with_building_density2 (3).csv
/kaggle/input/building-density2-500m-v2/train_df_with_building_density2 (3).csv
/kaggle/input/satellite-400m/validation_data_with_satellite.csv
/kaggle/input/satellite-400m/training_data_with_satellite.csv


In [2]:
# Feature Engineering
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.model_selection import train_test_split

# Machine Learning
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, make_scorer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor

import optuna

# Combine Datasets

In [3]:
train_density = pd.read_csv("/kaggle/input/data-with-density-200m/uncorrupted_training_data_with_density.csv")
train_density.head()

,Longitude,Latitude,datetime,UHI Index,density
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,8
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,8
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,8
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,8
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,8


In [4]:
train_satellite = pd.read_csv("/kaggle/input/satellite-400m/training_data_with_satellite.csv")
train_satellite.head()

,Longitude,Latitude,datetime,UHI Index,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B11,B12,NDVI,NDBI,NDWI,LST
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,1036.934045,1144.664522,1325.140099,1373.032466,1644.462855,2236.069319,2455.611436,2466.095934,2552.886809,2305.273326,1920.796870,0.284717,-0.033706,-0.300946,37.143640
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,1036.971023,1144.358700,1325.462754,1372.924045,1645.394702,2238.420167,2458.660325,2469.320650,2556.644227,2309.511781,1923.721499,0.285353,-0.033441,-0.301429,37.147602
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,1038.801550,1145.059082,1327.088037,1374.093302,1649.179292,2243.552647,2464.690699,2474.530126,2563.074729,2316.142001,1928.230915,0.285930,-0.033062,-0.301830,37.169067
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,1040.573078,1146.855305,1329.966823,1376.344928,1653.159602,2249.616779,2471.592663,2482.465069,2571.105817,2323.466530,1933.006431,0.286648,-0.033084,-0.302300,37.180983
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,1041.853259,1149.656095,1333.705642,1380.476761,1657.821397,2255.589594,2477.846536,2488.394475,2578.136510,2331.205495,1938.843467,0.286367,-0.032615,-0.302108,37.173729


In [5]:
train_satellite = train_satellite.drop(["Longitude","Latitude","datetime","UHI Index"], axis=1)
train_concat = pd.concat([train_density, train_satellite], axis = 1)
train_concat.head()

,Longitude,Latitude,datetime,UHI Index,density,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B11,B12,NDVI,NDBI,NDWI,LST
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,8,1036.934045,1144.664522,1325.140099,1373.032466,1644.462855,2236.069319,2455.611436,2466.095934,2552.886809,2305.273326,1920.796870,0.284717,-0.033706,-0.300946,37.143640
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,8,1036.971023,1144.358700,1325.462754,1372.924045,1645.394702,2238.420167,2458.660325,2469.320650,2556.644227,2309.511781,1923.721499,0.285353,-0.033441,-0.301429,37.147602
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,8,1038.801550,1145.059082,1327.088037,1374.093302,1649.179292,2243.552647,2464.690699,2474.530126,2563.074729,2316.142001,1928.230915,0.285930,-0.033062,-0.301830,37.169067
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,8,1040.573078,1146.855305,1329.966823,1376.344928,1653.159602,2249.616779,2471.592663,2482.465069,2571.105817,2323.466530,1933.006431,0.286648,-0.033084,-0.302300,37.180983
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,8,1041.853259,1149.656095,1333.705642,1380.476761,1657.821397,2255.589594,2477.846536,2488.394475,2578.136510,2331.205495,1938.843467,0.286367,-0.032615,-0.302108,37.173729


In [6]:
validation_density = pd.read_csv("/kaggle/input/data-with-density-200m/validation_data_with_density.csv")
validation_density.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Longitude,Latitude,UHI Index,density
0,-73.971665,40.788763,NaN,12
1,-73.971928,40.788875,NaN,12
2,-73.967080,40.789080,NaN,4
3,-73.972550,40.789082,NaN,12
4,-73.969697,40.787953,NaN,9


In [7]:
validation_satellite = pd.read_csv("/kaggle/input/satellite-400m/validation_data_with_satellite.csv")
validation_satellite.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Longitude,Latitude,UHI Index,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B11,B12,NDVI,NDBI,NDWI,LST
0,-73.971665,40.788763,NaN,1109.576929,1229.599619,1355.112575,1425.643830,1626.311228,2012.087688,2174.133802,2140.140682,2224.516030,2146.136730,1826.886547,0.200376,0.001399,-0.224598,36.637337
1,-73.971928,40.788875,NaN,1115.170029,1239.381140,1365.236988,1438.272807,1636.686696,2007.709649,2165.796345,2131.762135,2214.216520,2150.349123,1836.398977,0.194253,0.004341,-0.219195,36.703919
2,-73.967080,40.789080,NaN,795.622951,881.664959,1005.529128,972.993999,1216.328747,1958.375732,2245.151347,2245.597043,2336.408811,1815.682231,1377.212968,0.395391,-0.105857,-0.381427,33.485227
3,-73.972550,40.789082,NaN,1120.069213,1239.884109,1364.432543,1438.773632,1638.771291,2003.329968,2158.598186,2126.738806,2206.887035,2147.318115,1831.670618,0.192950,0.004815,-0.218353,36.817434
4,-73.969697,40.787953,NaN,951.219441,1046.451471,1173.158542,1190.453960,1423.341238,2052.694042,2301.973503,2288.260430,2383.179476,2009.478846,1600.800322,0.315578,-0.064867,-0.322152,35.072928


In [8]:
validation_satellite = validation_satellite.drop(["Longitude","Latitude","UHI Index"], axis=1)
validation_concat = pd.concat([validation_density, validation_satellite], axis=1)
validation_concat.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Longitude,Latitude,UHI Index,density,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B11,B12,NDVI,NDBI,NDWI,LST
0,-73.971665,40.788763,NaN,12,1109.576929,1229.599619,1355.112575,1425.643830,1626.311228,2012.087688,2174.133802,2140.140682,2224.516030,2146.136730,1826.886547,0.200376,0.001399,-0.224598,36.637337
1,-73.971928,40.788875,NaN,12,1115.170029,1239.381140,1365.236988,1438.272807,1636.686696,2007.709649,2165.796345,2131.762135,2214.216520,2150.349123,1836.398977,0.194253,0.004341,-0.219195,36.703919
2,-73.967080,40.789080,NaN,4,795.622951,881.664959,1005.529128,972.993999,1216.328747,1958.375732,2245.151347,2245.597043,2336.408811,1815.682231,1377.212968,0.395391,-0.105857,-0.381427,33.485227
3,-73.972550,40.789082,NaN,12,1120.069213,1239.884109,1364.432543,1438.773632,1638.771291,2003.329968,2158.598186,2126.738806,2206.887035,2147.318115,1831.670618,0.192950,0.004815,-0.218353,36.817434
4,-73.969697,40.787953,NaN,9,951.219441,1046.451471,1173.158542,1190.453960,1423.341238,2052.694042,2301.973503,2288.260430,2383.179476,2009.478846,1600.800322,0.315578,-0.064867,-0.322152,35.072928


# Add features

## 1. Road Density

In [9]:
train_road = pd.read_csv("/kaggle/input/new-road-400m/train_df_with_road_density (2).csv")
train_road.head()

,Longitude,Latitude,datetime,UHI Index,road_density
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,798.078274
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,796.366736
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,793.343813
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,790.201109
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,787.450494


In [10]:
validation_road = pd.read_csv("/kaggle/input/new-road-400m/valid_df_with_road_density (2).csv")
validation_road.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Longitude,Latitude,UHI Index,road_density
0,-73.971665,40.788763,NaN,1876.839924
1,-73.971928,40.788875,NaN,2095.162872
2,-73.967080,40.789080,NaN,1285.873617
3,-73.972550,40.789082,NaN,2848.513646
4,-73.969697,40.787953,NaN,1908.523907


In [11]:
train_road = train_road.drop(["Longitude","Latitude","datetime","UHI Index"], axis=1)
train_concat3 = pd.concat([train_concat, train_road], axis = 1)
train_concat3.head()

,Longitude,Latitude,datetime,UHI Index,density,B01,B02,B03,B04,B05,...,B07,B08,B8A,B11,B12,NDVI,NDBI,NDWI,LST,road_density
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,8,1036.934045,1144.664522,1325.140099,1373.032466,1644.462855,...,2455.611436,2466.095934,2552.886809,2305.273326,1920.796870,0.284717,-0.033706,-0.300946,37.143640,798.078274
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,8,1036.971023,1144.358700,1325.462754,1372.924045,1645.394702,...,2458.660325,2469.320650,2556.644227,2309.511781,1923.721499,0.285353,-0.033441,-0.301429,37.147602,796.366736
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,8,1038.801550,1145.059082,1327.088037,1374.093302,1649.179292,...,2464.690699,2474.530126,2563.074729,2316.142001,1928.230915,0.285930,-0.033062,-0.301830,37.169067,793.343813
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,8,1040.573078,1146.855305,1329.966823,1376.344928,1653.159602,...,2471.592663,2482.465069,2571.105817,2323.466530,1933.006431,0.286648,-0.033084,-0.302300,37.180983,790.201109
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,8,1041.853259,1149.656095,1333.705642,1380.476761,1657.821397,...,2477.846536,2488.394475,2578.136510,2331.205495,1938.843467,0.286367,-0.032615,-0.302108,37.173729,787.450494


In [12]:
train_concat3.shape

(11229, 21)

In [13]:
validation_road = validation_road.drop(["Longitude","Latitude","UHI Index"], axis=1)
validation_concat3 = pd.concat([validation_concat, validation_road], axis = 1)
validation_concat3.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Longitude,Latitude,UHI Index,density,B01,B02,B03,B04,B05,B06,B07,B08,B8A,B11,B12,NDVI,NDBI,NDWI,LST,road_density
0,-73.971665,40.788763,NaN,12,1109.576929,1229.599619,1355.112575,1425.643830,1626.311228,2012.087688,2174.133802,2140.140682,2224.516030,2146.136730,1826.886547,0.200376,0.001399,-0.224598,36.637337,1876.839924
1,-73.971928,40.788875,NaN,12,1115.170029,1239.381140,1365.236988,1438.272807,1636.686696,2007.709649,2165.796345,2131.762135,2214.216520,2150.349123,1836.398977,0.194253,0.004341,-0.219195,36.703919,2095.162872
2,-73.967080,40.789080,NaN,4,795.622951,881.664959,1005.529128,972.993999,1216.328747,1958.375732,2245.151347,2245.597043,2336.408811,1815.682231,1377.212968,0.395391,-0.105857,-0.381427,33.485227,1285.873617
3,-73.972550,40.789082,NaN,12,1120.069213,1239.884109,1364.432543,1438.773632,1638.771291,2003.329968,2158.598186,2126.738806,2206.887035,2147.318115,1831.670618,0.192950,0.004815,-0.218353,36.817434,2848.513646
4,-73.969697,40.787953,NaN,9,951.219441,1046.451471,1173.158542,1190.453960,1423.341238,2052.694042,2301.973503,2288.260430,2383.179476,2009.478846,1600.800322,0.315578,-0.064867,-0.322152,35.072928,1908.523907


In [14]:
validation_concat3.shape

(1040, 20)

## 2. Building density 2

In [15]:
train_density2 = pd.read_csv("/kaggle/input/building-density2-500m-v2/train_df_with_building_density2 (3).csv")
train_density2.head()

,Longitude,Latitude,datetime,UHI Index,building_density2
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,6
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,6
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,6
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,5
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,4


In [16]:
validation_density2 = pd.read_csv("/kaggle/input/building-density2-500m-v2/valid_df_with_building_density2 (3).csv")
validation_density2.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Longitude,Latitude,UHI Index,building_density2
0,-73.971665,40.788763,NaN,94
1,-73.971928,40.788875,NaN,97
2,-73.967080,40.789080,NaN,48
3,-73.972550,40.789082,NaN,94
4,-73.969697,40.787953,NaN,73


In [17]:
train_density2 = train_density2.drop(["Longitude","Latitude","datetime","UHI Index"], axis=1)
train_concat4 = pd.concat([train_concat3, train_density2], axis = 1)
train_concat4.head()

,Longitude,Latitude,datetime,UHI Index,density,B01,B02,B03,B04,B05,...,B08,B8A,B11,B12,NDVI,NDBI,NDWI,LST,road_density,building_density2
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,8,1036.934045,1144.664522,1325.140099,1373.032466,1644.462855,...,2466.095934,2552.886809,2305.273326,1920.796870,0.284717,-0.033706,-0.300946,37.143640,798.078274,6
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,8,1036.971023,1144.358700,1325.462754,1372.924045,1645.394702,...,2469.320650,2556.644227,2309.511781,1923.721499,0.285353,-0.033441,-0.301429,37.147602,796.366736,6
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,8,1038.801550,1145.059082,1327.088037,1374.093302,1649.179292,...,2474.530126,2563.074729,2316.142001,1928.230915,0.285930,-0.033062,-0.301830,37.169067,793.343813,6
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,8,1040.573078,1146.855305,1329.966823,1376.344928,1653.159602,...,2482.465069,2571.105817,2323.466530,1933.006431,0.286648,-0.033084,-0.302300,37.180983,790.201109,5
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,8,1041.853259,1149.656095,1333.705642,1380.476761,1657.821397,...,2488.394475,2578.136510,2331.205495,1938.843467,0.286367,-0.032615,-0.302108,37.173729,787.450494,4


In [18]:
train_concat4.shape

(11229, 22)

In [19]:
validation_density2 = validation_density2.drop(["Longitude","Latitude","UHI Index"], axis=1)
validation_concat4 = pd.concat([validation_concat3, validation_density2], axis = 1)
validation_concat4.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,Longitude,Latitude,UHI Index,density,B01,B02,B03,B04,B05,B06,...,B08,B8A,B11,B12,NDVI,NDBI,NDWI,LST,road_density,building_density2
0,-73.971665,40.788763,NaN,12,1109.576929,1229.599619,1355.112575,1425.643830,1626.311228,2012.087688,...,2140.140682,2224.516030,2146.136730,1826.886547,0.200376,0.001399,-0.224598,36.637337,1876.839924,94
1,-73.971928,40.788875,NaN,12,1115.170029,1239.381140,1365.236988,1438.272807,1636.686696,2007.709649,...,2131.762135,2214.216520,2150.349123,1836.398977,0.194253,0.004341,-0.219195,36.703919,2095.162872,97
2,-73.967080,40.789080,NaN,4,795.622951,881.664959,1005.529128,972.993999,1216.328747,1958.375732,...,2245.597043,2336.408811,1815.682231,1377.212968,0.395391,-0.105857,-0.381427,33.485227,1285.873617,48
3,-73.972550,40.789082,NaN,12,1120.069213,1239.884109,1364.432543,1438.773632,1638.771291,2003.329968,...,2126.738806,2206.887035,2147.318115,1831.670618,0.192950,0.004815,-0.218353,36.817434,2848.513646,94
4,-73.969697,40.787953,NaN,9,951.219441,1046.451471,1173.158542,1190.453960,1423.341238,2052.694042,...,2288.260430,2383.179476,2009.478846,1600.800322,0.315578,-0.064867,-0.322152,35.072928,1908.523907,73


In [20]:
validation_concat4.shape

(1040, 21)

## 3. Building density 3

In [21]:
train_density3 = pd.read_csv("/kaggle/input/building-density3-15-20m-200m/train_df_with_building_density3 (6).csv")
train_density3.head()

,Longitude,Latitude,datetime,UHI Index,building_density3
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,14
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,12
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,13
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,14
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,14


In [22]:
validation_density3 = pd.read_csv("/kaggle/input/building-density3-15-20m-200m/valid_df_with_building_density3 (6).csv")
validation_density3.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,Longitude,Latitude,UHI Index,building_density3
0,-73.971665,40.788763,NaN,26
1,-73.971928,40.788875,NaN,24
2,-73.967080,40.789080,NaN,29
3,-73.972550,40.789082,NaN,29
4,-73.969697,40.787953,NaN,157


In [23]:
train_density3 = train_density3.drop(["Longitude","Latitude","datetime","UHI Index"], axis=1)
train_concat5 = pd.concat([train_concat4, train_density3], axis = 1)
train_concat5.head()

,Longitude,Latitude,datetime,UHI Index,density,B01,B02,B03,B04,B05,...,B8A,B11,B12,NDVI,NDBI,NDWI,LST,road_density,building_density2,building_density3
0,-73.909167,40.813107,24-07-2021 15:53,1.030289,8,1036.934045,1144.664522,1325.140099,1373.032466,1644.462855,...,2552.886809,2305.273326,1920.796870,0.284717,-0.033706,-0.300946,37.143640,798.078274,6,14
1,-73.909187,40.813045,24-07-2021 15:53,1.030289,8,1036.971023,1144.358700,1325.462754,1372.924045,1645.394702,...,2556.644227,2309.511781,1923.721499,0.285353,-0.033441,-0.301429,37.147602,796.366736,6,12
2,-73.909215,40.812978,24-07-2021 15:53,1.023798,8,1038.801550,1145.059082,1327.088037,1374.093302,1649.179292,...,2563.074729,2316.142001,1928.230915,0.285930,-0.033062,-0.301830,37.169067,793.343813,6,13
3,-73.909242,40.812908,24-07-2021 15:53,1.023798,8,1040.573078,1146.855305,1329.966823,1376.344928,1653.159602,...,2571.105817,2323.466530,1933.006431,0.286648,-0.033084,-0.302300,37.180983,790.201109,5,14
4,-73.909257,40.812845,24-07-2021 15:53,1.021634,8,1041.853259,1149.656095,1333.705642,1380.476761,1657.821397,...,2578.136510,2331.205495,1938.843467,0.286367,-0.032615,-0.302108,37.173729,787.450494,4,14


In [24]:
train_concat5.shape

(11229, 23)

In [25]:
validation_density3 = validation_density3.drop(["Longitude","Latitude","UHI Index"], axis=1)
validation_concat5 = pd.concat([validation_concat4, validation_density3], axis = 1)
validation_concat5.head()

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pan

,Longitude,Latitude,UHI Index,density,B01,B02,B03,B04,B05,B06,...,B8A,B11,B12,NDVI,NDBI,NDWI,LST,road_density,building_density2,building_density3
0,-73.971665,40.788763,NaN,12,1109.576929,1229.599619,1355.112575,1425.643830,1626.311228,2012.087688,...,2224.516030,2146.136730,1826.886547,0.200376,0.001399,-0.224598,36.637337,1876.839924,94,26
1,-73.971928,40.788875,NaN,12,1115.170029,1239.381140,1365.236988,1438.272807,1636.686696,2007.709649,...,2214.216520,2150.349123,1836.398977,0.194253,0.004341,-0.219195,36.703919,2095.162872,97,24
2,-73.967080,40.789080,NaN,4,795.622951,881.664959,1005.529128,972.993999,1216.328747,1958.375732,...,2336.408811,1815.682231,1377.212968,0.395391,-0.105857,-0.381427,33.485227,1285.873617,48,29
3,-73.972550,40.789082,NaN,12,1120.069213,1239.884109,1364.432543,1438.773632,1638.771291,2003.329968,...,2206.887035,2147.318115,1831.670618,0.192950,0.004815,-0.218353,36.817434,2848.513646,94,29
4,-73.969697,40.787953,NaN,9,951.219441,1046.451471,1173.158542,1190.453960,1423.341238,2052.694042,...,2383.179476,2009.478846,1600.800322,0.315578,-0.064867,-0.322152,35.072928,1908.523907,73,157


In [26]:
validation_concat5.shape

(1040, 22)

# Select features 

In [27]:
features = ['B01','B8A','B11','B12','LST','NDVI','NDBI','NDWI','density','road_density','building_density2','building_density3']
train_df = train_concat5[features + ["UHI Index"]]
train_df

,B01,B8A,B11,B12,LST,NDVI,NDBI,NDWI,density,road_density,building_density2,building_density3,UHI Index
0,1036.934045,2552.886809,2305.273326,1920.796870,37.143640,0.284717,-0.033706,-0.300946,8,798.078274,6,14,1.030289
1,1036.971023,2556.644227,2309.511781,1923.721499,37.147602,0.285353,-0.033441,-0.301429,8,796.366736,6,12,1.030289
2,1038.801550,2563.074729,2316.142001,1928.230915,37.169067,0.285930,-0.033062,-0.301830,8,793.343813,6,13,1.023798
3,1040.573078,2571.105817,2323.466530,1933.006431,37.180983,0.286648,-0.033084,-0.302300,8,790.201109,5,14,1.023798
4,1041.853259,2578.136510,2331.205495,1938.843467,37.173729,0.286367,-0.032615,-0.302108,8,787.450494,4,14,1.021634
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11224,505.685643,3164.168008,1954.692083,1197.219230,31.250745,0.643753,-0.211530,-0.590693,0,1579.040587,26,0,0.972470
11225,506.378600,3154.576962,1951.042245,1196.335477,31.230093,0.642562,-0.210976,-0.589601,0,1586.321518,26,0,0.972470
11226,506.808327,3137.486194,1943.507378,1193.436669,31.220690,0.641328,-0.210249,-0.588194,0,1595.356778,26,0,0.981124
11227,506.673153,3134.157718,1941.849744,1192.670373,31.205963,0.640902,-0.210145,-0.587743,0,1598.145380,26,0,0.981245


# Remove duplicates from training data

In [28]:
# Remove duplicate rows from the DataFrame based on specified columns and keep the first occurrence
for col in features:
    # Check if the value is a numpy array and has more than one dimension
    train_df[col] = train_df[col].apply(lambda x: tuple(x) if isinstance(x, np.ndarray) and x.ndim > 0 else x)

# Now remove duplicates
uhi_data = train_df.drop_duplicates(subset=features, keep='first')
uhi_data.head()

<ipython-input-28-cc52a2d8b8e0>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[col] = train_df[col].apply(lambda x: tuple(x) if isinstance(x, np.ndarray) and x.ndim > 0 else x)


,B01,B8A,B11,B12,LST,NDVI,NDBI,NDWI,density,road_density,building_density2,building_density3,UHI Index
0,1036.934045,2552.886809,2305.273326,1920.796870,37.143640,0.284717,-0.033706,-0.300946,8,798.078274,6,14,1.030289
1,1036.971023,2556.644227,2309.511781,1923.721499,37.147602,0.285353,-0.033441,-0.301429,8,796.366736,6,12,1.030289
2,1038.801550,2563.074729,2316.142001,1928.230915,37.169067,0.285930,-0.033062,-0.301830,8,793.343813,6,13,1.023798
3,1040.573078,2571.105817,2323.466530,1933.006431,37.180983,0.286648,-0.033084,-0.302300,8,790.201109,5,14,1.023798
4,1041.853259,2578.136510,2331.205495,1938.843467,37.173729,0.286367,-0.032615,-0.302108,8,787.450494,4,14,1.021634


In [29]:
uhi_data.shape

(11229, 13)

In [30]:
# Resetting the index of the dataset
uhi_data=uhi_data.reset_index(drop=True)

In [31]:
uhi_data.isna().sum()

B01                  0
B8A                  0
B11                  0
B12                  0
LST                  0
NDVI                 0
NDBI                 0
NDWI                 0
density              0
road_density         0
building_density2    0
building_density3    0
UHI Index            0
dtype: int64

# Model Building

In [32]:
# Split the data into features (X) and target (y), and then into training and testing sets
X = uhi_data.drop(columns=['UHI Index']).values
y = uhi_data ['UHI Index'].values
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.01,random_state=42)

**Feature Scaling**

In [33]:
# Scale the training and test data using standardscaler
sc = StandardScaler()
X = sc.fit_transform(X)
#X_test = sc.transform(X_test)

**Model Training**

## XGBoost

In [34]:
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 1200, step=100),
        'max_depth': trial.suggest_int('max_depth', 2, 120, step=2),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0, 0.8),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-5, 10),  # L1 regularization
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-5, 10) # L2 regularization
    }
    
    model = XGBRegressor(objective='reg:squarederror', random_state=42, **params)
    score = cross_val_score(model, X, y, cv=5, scoring='r2', n_jobs=-1)
    return score.mean()

# Run the optimization
#study = optuna.create_study(direction='maximize')
#study.optimize(objective, n_trials=10000, n_jobs=-1)

# Best hyperparameters
#best_params = study.best_params
#print("Best Hyperparameters found:", best_params)

# 📌 Version 0.9766
best_params = {'n_estimators': 1100, 'max_depth': 114, 'learning_rate': 0.015591888704234214, 'subsample': 0.8049957937585521, 'colsample_bytree': 0.8908642503612316, 'min_child_weight': 4, 'gamma': 1.923273380075315e-07, 'reg_alpha': 1.766675566559426e-05, 'reg_lambda': 1.148751628522958}
# Train the model with best parameters
best_xgb = XGBRegressor(**best_params)
#best_xgb.fit(X, y)

## CatBoost

### Define validation function

In [35]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import make_pipeline

def cross_validation(model, X, y, cv, verbose=True):
    kf = KFold(n_splits=cv, shuffle=True, random_state=42)
    pipeline = make_pipeline(StandardScaler(), model)
    scores = cross_val_score(model, X, y, scoring="r2", cv=kf)
    if verbose:
        model_name = model.__class__.__name__
        print(f"Cross validation score for {model_name}: {scores.mean():.3f} +- {scores.std():.3f}")
    return scores

In [36]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning)

from sklearn.metrics import mean_squared_error

def custom_fit_cross_val(model, X, y, cv):
    kf = KFold(n_splits=cv, shuffle=True, random_state=42) 
    cv_scores = []

    for train_idx, val_idx in kf.split(X):
        X_train_cv, X_val_cv = X[train_idx], X[val_idx]
        y_train_cv, y_val_cv = y[train_idx], y[val_idx]

        sc = StandardScaler()
        sc.fit_transform(X_train_cv)
        sc.transform(X_val_cv)
        
        # Fit with early stopping
        model.fit(
            X_train_cv, y_train_cv,
            eval_set=[(X_val_cv, y_val_cv)], 
            early_stopping_rounds=50, 
            verbose=False
        )
            
        y_pred_cv = model.predict(X_val_cv)
        
        score = mean_squared_error(y_val_cv, y_pred_cv, squared=False)
        cv_scores.append(score)

    return np.mean(cv_scores)

In [37]:
from catboost import CatBoostRegressor

def objective(trial):
    boosting_type = trial.suggest_categorical('boosting_type', ['Ordered', 'Plain'])
    grow_policy = trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide'])

    all_score_functions = ['SolarL2', 'Cosine', 'NewtonCosine', 'L2', 'LOOL2', 'NewtonL2']
    score_function = trial.suggest_categorical('score_function', all_score_functions)

    if boosting_type == 'Ordered' and score_function in ['LOOL2', 'SolarL2', 'L2', 'NewtonL2']:
        return float('inf')

    if boosting_type == 'Ordered' and grow_policy in ['Lossguide', 'Depthwise']:
        return float('inf')

    params = {
        'boosting_type': boosting_type,
        'grow_policy': grow_policy,
        "score_function": score_function,
        "iterations": trial.suggest_int("iterations", 100, 1500),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "depth": trial.suggest_int("depth", 3, 12),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 0.01, 10.0, log=True),
        "border_count": trial.suggest_int("border_count", 32, 255),
        "random_strength": trial.suggest_float("random_strength", 1.0, 10.0),
        "leaf_estimation_iterations": trial.suggest_int("leaf_estimation_iterations", 1, 10),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 1, 50),
        'feature_border_type': trial.suggest_categorical('feature_border_type', ['GreedyLogSum', 'MinEntropy', 'Median', 'UniformAndQuantiles']),
        'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bayesian', 'Bernoulli', 'No', 'Poisson']),
        "eval_metric": "RMSE",
        "loss_function": "RMSE",
        "task_type": "GPU",
        "verbose": 0,
    }

        # Conditional parameters based on the selected settings
    if params['grow_policy'] == 'Lossguide':
        params['max_leaves'] = trial.suggest_int('max_leaves', 2, 64)

    if params['bootstrap_type'] in ['Bernoulli', 'Poisson']:
        params['subsample'] = trial.suggest_float('subsample', 0.1, 1.0)

    if params['bootstrap_type'] == 'Bayesian':
        params['bagging_temperature'] = trial.suggest_float('bagging_temperature', 0, 10)
    
    
    model = CatBoostRegressor(**params)
    return custom_fit_cross_val(model, X, y, cv=5)

# Run Optuna optimization
#study_cb = optuna.create_study(direction="minimize")
#study_cb.optimize(objective, n_trials=100)

# 5. Print best hyperparameters
#best_params = study_cb.best_params
#best_params["task_type"] = "GPU"
#best_params["eval_metric"] = "RMSE"
#best_params["loss_function"] = "RMSE"
#best_params["verbose"] = 0
#print("Best CatBoost Hyperparameters:", best_params)
#print("Best rmse score is:", study_cb.best_value)

# Train the model with best parameters
# 📌 Version 0.9750
best_params = {'boosting_type': 'Plain', 'grow_policy': 'Depthwise', 'score_function': 'SolarL2', 'iterations': 1039, 'learning_rate': 0.05627818345201275, 'depth': 10, 'l2_leaf_reg': 8.330279687259916, 'border_count': 255, 'random_strength': 7.767297165881897, 'leaf_estimation_iterations': 4, 'min_data_in_leaf': 13, 'feature_border_type': 'UniformAndQuantiles', 'bootstrap_type': 'Bayesian', 'bagging_temperature': 1.2397344481053751}
best_params["task_type"] = "GPU"
best_params["eval_metric"] = "RMSE"
best_params["loss_function"] = "RMSE"
best_params["verbose"] = 0
best_cb = CatBoostRegressor(**best_params)
# best_cb.fit(X,y)

## HistGradientBoosting

In [38]:
from sklearn.ensemble import HistGradientBoostingRegressor

def objective(trial):
    # Define hyperparameters to tune
    params = {
        "max_iter": trial.suggest_int('max_iter', 50, 1000),
        "learning_rate": trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        "max_leaf_nodes": trial.suggest_int('max_leaf_nodes', 10, 200),
        "min_samples_leaf": trial.suggest_int('min_samples_leaf', 1, 20),
        "l2_regularization": trial.suggest_float('l2_regularization', 1e-4, 10.0, log=True),
        "max_bins": trial.suggest_int('max_bins', 16, 255),
        "max_depth": trial.suggest_int('max_depth', 3, 20),
        "scoring": "r2",
        "random_state": 42,
    }

    # Create a HistGradientBoostingRegressor model with the suggested hyperparameters
    model = HistGradientBoostingRegressor(**params)
    scores = cross_validation(model, X, y, cv=5, verbose=False)
    return scores.mean() 

# # Create an Optuna study and optimize
#study_histgb = optuna.create_study(direction='maximize') 
#study_histgb.optimize(objective, n_trials=200)

#best_params = study_histgb.best_params
# Print the best hyperparameters
#print("Best HistGB hyperparameters:", study_histgb.best_params)
#print("Best cross validation score is:", study_histgb.best_value)

# Train the model with best parameters
# Version 0.????
best_params = {'max_iter': 616, 'learning_rate': 0.148212634641663, 'max_leaf_nodes': 197, 'min_samples_leaf': 11, 'l2_regularization': 9.796190958525132, 'max_bins': 130, 'max_depth': 19}
best_hgb = HistGradientBoostingRegressor(**best_params)
# best_hgb.fit(X,y)

In [39]:
#best_hgb = HistGradientBoostingRegressor(**best_params)
#best_hgb.fit(X, y)

## LightGBM

In [40]:
pip install optuna-integration[lightgbm]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.6/97.6 kB 3.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from lightgbm import early_stopping
from optuna.integration import LightGBMPruningCallback

def lgbm_cross_val(model, trial, X, y, cv):
    kf = KFold(n_splits=cv, shuffle=True, random_state=42) 
    cv_scores = []

    for train_idx, val_idx in kf.split(X):
        X_train_cv, X_val_cv = X[train_idx], X[val_idx]
        y_train_cv, y_val_cv = y[train_idx], y[val_idx]

        sc = StandardScaler()
        sc.fit_transform(X_train_cv)
        sc.transform(X_val_cv)
        
        # Fit with early stopping
        model.fit(
            X_train_cv, y_train_cv,
            eval_set=[(X_val_cv, y_val_cv)], 
            callbacks=[early_stopping(50), LightGBMPruningCallback(trial, "rmse"),]
        )
            
        y_pred_cv = model.predict(X_val_cv)
        
        score = mean_squared_error(y_val_cv, y_pred_cv, squared=False)
        cv_scores.append(score)

    return np.mean(cv_scores)

In [ ]:
from lightgbm import LGBMRegressor

def objective(trial):
    boosting_type = trial.suggest_categorical("boosting_type", ["gbdt", "dart", "goss", "rf"])
    params = {
        "boosting_type": boosting_type,
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),  # Number of trees
        "learning_rate": trial.suggest_float("learning_rate", 0.005, 0.5, log=True),  # Step size shrinkage
        "num_leaves": trial.suggest_int("num_leaves", 10, 200),  # Maximum number of leaves per tree
        "max_depth": trial.suggest_int("max_depth", 3, 20),  # Depth of trees
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),  # Minimum samples per leaf
        "feature_fraction": trial.suggest_float("feature_fraction", 0.5, 1.0),  # Fraction of features per tree
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-4, 10.0, log=True),  # L1 regularization
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-4, 10.0, log=True),  # L2 regularization
        "metric": "rmse",
        # "device_type": "gpu",
        "random_state": 42,
        "verbose": -1,
    }

    # Mode-specific tuning
    if boosting_type == "gbdt" or boosting_type == "rf":
        params["bagging_fraction"] = trial.suggest_float("bagging_fraction", 0.5, 1.0)
        params["bagging_freq"] = trial.suggest_int("bagging_freq", 1, 10)
    
    elif boosting_type == "dart":
        params["drop_rate"] = trial.suggest_float("drop_rate", 0.01, 0.5)
        params["max_drop"] = trial.suggest_int("max_drop", 5, 50)
        params["skip_drop"] = trial.suggest_float("skip_drop", 0.0, 1.0)
    
    elif boosting_type == "goss":
        params["top_rate"] = trial.suggest_float("top_rate", 0.1, 0.5)
        params["other_rate"] = trial.suggest_float("other_rate", 0.1, 0.5)
        # No bagging_fraction in GOSS
    
    model = LGBMRegressor(**params)
    return lgbm_cross_val(model, trial, X, y, cv=5)

# #  4. Run Optuna optimization
#study_lgb = optuna.create_study(direction="minimize") 
#study_lgb.optimize(objective, n_trials=5000)

#best_params = study_lgb.best_params
# #  5. Print best hyperparameters
#print("Best LightGBM Hyperparameters:", study_lgb.best_params)
#print("Best cross validation score is:", study_lgb.best_value)

In [ ]:
# 📌 Version 0.9741
best_params = {'boosting_type': 'goss', 'n_estimators': 936, 'learning_rate': 0.21380353035497862, 'num_leaves': 186, 'max_depth': 18, 'min_data_in_leaf': 3, 'feature_fraction': 0.9876949484780569, 'lambda_l1': 0.010202698630371546, 'lambda_l2': 2.489485764075577, 'top_rate': 0.25542235514965383, 'other_rate': 0.36917949534546296}
best_lgbm = LGBMRegressor(**best_params)
# Version 0.9723
#best_params = {'boosting_type': 'gbdt', 'n_estimators': 706, 'learning_rate': 0.13863348040981266, 'num_leaves': 195, 'max_depth': 17, 'min_data_in_leaf': 23, 'feature_fraction': 0.9984228159953552, 'lambda_l1': 0.00012149732328269774, 'lambda_l2': 0.00022579481350169068, 'bagging_fraction': 0.7514501921889513, 'bagging_freq': 6}
#best_lgbm = LGBMRegressor(**best_params)
#best_lgbm.fit(X, y)

## AdaBoost

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

# Define the objective function for Optuna
def objective(trial):
    # Define hyperparameters to tune
    n_estimators = trial.suggest_int('n_estimators', 50, 500)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 1.0, log=True)
    loss = trial.suggest_categorical('loss', ['linear', 'square', 'exponential'])

    # Tune hyperparameters of the base estimator (DecisionTreeRegressor)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    # Create a base estimator with the suggested hyperparameters
    base_estimator = DecisionTreeRegressor(
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42
    )
    # Create an AdaBoostRegressor model with the suggested hyperparameters
    model = AdaBoostRegressor(
        estimator=base_estimator,
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        loss=loss,
        random_state=42
    )


    scores = cross_validation(model, X, y, cv=5, verbose=False)
    return scores.mean() 

# # Create an Optuna study and optimize
#study_ada = optuna.create_study(direction='maximize')  # Maximize R² score
#study_ada.optimize(objective, n_trials=3)

# # Print the best hyperparameters
#best_params = study_ada.best_params
# 0.9560 cv score
best_params = {'n_estimators': 173, 'learning_rate': 0.6399437769220059, 'loss': 'exponential', "estimator":DecisionTreeRegressor(max_depth=10, min_samples_split=2, min_samples_leaf=1)}
best_ada = AdaBoostRegressor(**best_params)
#print("Best AdaBoost hyperparameters:", best_params)
#print("Best cross validation score:", study_ada.best_value)

## Optuna Ensemble

In [ ]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_log_error

def genOOF(model, X, y, model_type):
    """
    Generate Out-of-Fold (OOF) predictions and evaluate models.

    Args:
        model: The model to train.
        X: Features dataset (numpy array or DataFrame).
        y: Target variable (continuous).
        model_type: Model type identifier (1 for XGBoost, 2 for CatBoost, 3 for HistGradientBoosting).

    Returns:
        oof_valid_preds: Out-of-fold predictions.
    """
    cv = KFold(n_splits=10, shuffle=True, random_state=42)  # Use KFold for continuous targets
    cv_scores = np.empty(10)
    oof_valid_preds = np.zeros(len(y))

    for fold, (train_idx, valid_idx) in enumerate(cv.split(X)):
        X_train, y_train = X[train_idx], y[train_idx]  # Training set for the current fold
        X_valid, y_valid = X[valid_idx], y[valid_idx]  # Validation set for the current fold

        if model_type == 1:  # XGBoost
            model.fit(X_train, y_train,
                      eval_set=[(X_valid, y_valid)],
                      early_stopping_rounds=20,
                      verbose=False)
        elif model_type == 2:  # CatBoost
            model.fit(X_train, y_train,
                      eval_set=(X_valid, y_valid),
                      verbose=False,
                      early_stopping_rounds=20)  
        elif model_type == 3:  # HistGradientBoosting
            model.fit(X_train, y_train)  
        elif model_type == 4: #LightGBM
            model.fit(X_train, y_train, 
                      eval_set=[(X_valid, y_valid)]) 
        elif model_type == 5: #AdaBoost
            model.fit(X_train, y_train) 

        valid_preds = model.predict(X_valid)
        oof_valid_preds[valid_idx] = valid_preds

        # Calculate the evaluation metric (RMSLE) for the validation set
        cv_scores[fold] = np.sqrt(mean_squared_log_error(y_valid, valid_preds))

    mean_rmse = np.mean(cv_scores)  # Average evaluation metric across all folds
    names = {1: "XGBoost", 2: "CatBoost", 3: "HistGradientBoosting", 4:"LightGBM", 5:"AdaBoost"}
    print(f'The mean RMSE for {names[model_type]} is {mean_rmse:.4f}')
    return oof_valid_preds

In [ ]:
oof_valid_preds_xgb = genOOF(best_xgb, X, y, 1)

In [ ]:
oof_valid_preds_cb = genOOF(best_cb, X, y, 2)

In [ ]:
#oof_valid_preds_hgb = genOOF(best_hgb, X, y, 3)

In [ ]:
oof_valid_preds_lgbm = genOOF(best_lgbm, X, y, 4)

In [ ]:
#oof_valid_preds_ada = genOOF(best_ada, X, y, 5)

In [ ]:
!pip install cmaes

In [ ]:
from functools import partial
import numpy as np
import optuna
from sklearn.metrics import r2_score

class OptunaWeights:
    def __init__(self, random_state, n_trials=10000):
        self.study = None  # Optuna study object
        self.weights = None  # Optimal weights for the predictions
        self.random_state = random_state  # Random state for reproducibility
        self.n_trials = n_trials  # Number of trials for hyperparameter optimization

    def _objective(self, trial, y_true, y_preds):
        # Define the weights for the predictions from each model
        weights = [trial.suggest_float(f"weight{n}", 0, 1) for n in range(len(y_preds))]
        total = sum(weights)
        weights = [x / total for x in weights]  # Ensure the sum of weights is 1
        
        # Calculate the weighted prediction
        weighted_pred = np.average(np.array(y_preds), axis=0, weights=weights)
        
        # Calculate the R² score
        r2 = r2_score(y_true, weighted_pred)
        
        # Return negative R² because Optuna minimizes by default and we want to maximize R²
        return -r2  # Negative because we want to maximize R²

    def fit(self, y_true, y_preds):
        # Set Optuna logging verbosity to ERROR
        optuna.logging.set_verbosity(optuna.logging.ERROR)
        
        # Create a CMA-ES sampler for hyperparameter optimization
        sampler = optuna.samplers.CmaEsSampler(seed=self.random_state)
        
        # Create a Hyperband pruner for early stopping
        pruner = optuna.pruners.HyperbandPruner()
        
        # Create an Optuna study with the specified sampler, pruner, and objective direction
        # Direction is 'minimize' because we're minimizing -R² (which maximizes R²)
        self.study = optuna.create_study(sampler=sampler, pruner=pruner, 
                                         study_name="OptunaWeights", direction='minimize')
        
        # Create a partial function for the objective with y_true and y_preds as fixed arguments
        objective_partial = partial(self._objective, y_true=y_true, y_preds=y_preds)
        
        # Optimize the study with the specified number of trials and show progress bar
        self.study.optimize(objective_partial, n_trials=self.n_trials, show_progress_bar=True)
        
        # Extract the best weights from the study's best parameters
        weights = [self.study.best_params[f"weight{n}"] for n in range(len(y_preds))]
        print("Weights before scaling:", weights)
        
        # Scale weights to ensure sum of weights is 1
        total = sum(weights)
        self.weights = [x / total for x in weights]
        print("Weights after scaling:", self.weights)
        
        # Calculate and print the best R² score achieved
        weighted_pred = np.average(np.array(y_preds), axis=0, weights=self.weights)
        best_r2 = r2_score(y_true, weighted_pred)
        print(f"Best R² score: {best_r2:.6f}")


ow = OptunaWeights(42)
ow.fit(y, y_preds=[oof_valid_preds_xgb, oof_valid_preds_cb, oof_valid_preds_lgbm])
weights = ow.weights
weights

# Submission

In [ ]:
# Train with the whole datasets
best_xgb.fit(X, y)
best_cb.fit(X, y)
#best_hgb.fit(X, y)
best_lgbm.fit(X, y)
#best_ada.fit(X, y)

In [ ]:
validation_concat5.head()

In [ ]:
submission_val_data = validation_concat5[features]
submission_val_data.head()

In [ ]:
# Feature Scaling 
submission_val_data = submission_val_data.values
transformed_submission_data = sc.transform(submission_val_data)

In [ ]:
#Making predictions
final_predictions1 = best_xgb.predict(transformed_submission_data)
final_predictions2 = best_cb.predict(transformed_submission_data)
#final_predictions3 = best_hgb.predict(transformed_submission_data)
final_predictions4 = best_lgbm.predict(transformed_submission_data)
#final_predictions5 = best_ada.predict(transformed_submission_data)

final_predictions = final_predictions1*weights[0] + final_predictions2*weights[1] + final_predictions4*weights[2] 
final_prediction_series = pd.Series(final_predictions)

In [ ]:
#Combining the results into dataframe
submission_df = pd.DataFrame({'Longitude':validation_concat['Longitude'].values, 'Latitude':validation_concat['Latitude'].values, 'UHI Index':final_prediction_series.values})

In [ ]:
#Displaying the sample submission dataframe
submission_df.head()

In [ ]:
#Dumping the predictions into a csv file.
submission_df.to_csv("submission.csv",index = False)